In [23]:
import requests
import duckdb
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import load_dataset, get_dataset_split_names # Huggingface datasets

In [9]:
r = requests.get("https://datasets-server.huggingface.co/parquet?dataset=blog_authorship_corpus")
j = r.json()
urls = [f['url'] for f in j['parquet_files'] if f['split'] == 'train']

In [10]:
urls

['https://huggingface.co/datasets/blog_authorship_corpus/resolve/refs%2Fconvert%2Fparquet/blog_authorship_corpus/blog_authorship_corpus-train-00000-of-00002.parquet',
 'https://huggingface.co/datasets/blog_authorship_corpus/resolve/refs%2Fconvert%2Fparquet/blog_authorship_corpus/blog_authorship_corpus-train-00001-of-00002.parquet']

In [5]:
url = "https://huggingface.co/datasets/blog_authorship_corpus/resolve/refs%2Fconvert%2Fparquet/blog_authorship_corpus/blog_authorship_corpus-train-00000-of-00002.parquet"

con = duckdb.connect()
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")

In [6]:
con.sql(f"""SELECT horoscope, 
    count(*), 
    AVG(LENGTH(text)) AS avg_blog_length 
    FROM '{url}' 
    GROUP BY horoscope 
    ORDER BY avg_blog_length 
    DESC LIMIT(5)"""
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────────┬──────────────┬────────────────────┐
│ horoscope │ count_star() │  avg_blog_length   │
│  varchar  │    int64     │       double       │
├───────────┼──────────────┼────────────────────┤
│ Aquarius  │        34062 │  1129.218836239798 │
│ Cancer    │        41509 │  1098.366812016671 │
│ Capricorn │        33961 │ 1073.2002002296751 │
│ Libra     │        40302 │ 1072.0718326633914 │
│ Leo       │        40587 │ 1064.0536871412028 │
└───────────┴──────────────┴────────────────────┘

In [13]:
url = "https://huggingface.co/datasets/birkhoffg/folktables-acs-income/blob/main/data/train-00000-of-00002-a71537175a4688ae.parquet"

con.sql(f"""SELECT
    count(*), 
    AVG(LENGTH(STATE)) AS avg_state_length
    FROM '{url}' 
    ORDER BY avg_state_length 
    DESC LIMIT(5)"""
)

RuntimeError: IO Error: HTTP GET error: Content-Length from server mismatches requested range, server may not support range requests.

## Load dataset

In [26]:
get_dataset_split_names("birkhoffg/folktables-acs-income")

['train', 'test']

In [15]:
dataset = load_dataset("birkhoffg/folktables-acs-income", split="train")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7345626 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2448543 [00:00<?, ? examples/s]

In [16]:
dataset

Dataset({
    features: ['AGEP', 'COW', 'SCHL', 'MAR', 'OCCP', 'POBP', 'RELP', 'WKHP', 'SEX', 'RAC1P', 'STATE', 'YEAR', 'PINCP', '__index_level_0__'],
    num_rows: 7345626
})

In [17]:
type(dataset)

datasets.arrow_dataset.Dataset

In [28]:
dataset[0]

{'AGEP': 29.0,
 'COW': 1.0,
 'SCHL': 16.0,
 'MAR': 1.0,
 'OCCP': 4920.0,
 'POBP': 12.0,
 'RELP': 1.0,
 'WKHP': 30.0,
 'SEX': 2.0,
 'RAC1P': 8.0,
 'STATE': 'FL',
 'YEAR': 2018,
 'PINCP': 0.0,
 '__index_level_0__': 98618}

In [32]:
# Connect to DuckDB
con = duckdb.connect()

In [33]:
# Create a table in DuckDB
con.execute("CREATE TABLE folktables (state VARCHAR, income INT)")

In [36]:
# Insert the data into the table
for row in dataset:
    con.execute(f"INSERT INTO folktables VALUES ('{row['STATE']}', {row['INCOME']})")

KeyError: 'INCOME'

In [37]:
# Query the data
result = con.execute("""
    SELECT COUNT(*)
    ,AVG(LENGTH(state)) AS avg_state_length
    FROM folktables
    ORDER BY avg_state_length DESC
    LIMIT 5
""")


In [38]:
result

In [20]:
dataset.to_arrow('data.arrow')

nyc = ds.dataset('nyc-taxi/', partitioning=["year", "month"])

AttributeError: 'Dataset' object has no attribute 'to_arrow'

In [19]:
duckdb.sql("""SELECT * FROM dataset LIMIT 10""")

InvalidInputException: Invalid Input Error: Python Object "dataset" of type "Dataset" found on line "/var/folders/g7/822123tx4gn6ndvw2jgh5f140000gn/T/ipykernel_99615/1421186819.py:1" not suitable for replacement scans.
Make sure that "dataset" is either a pandas.DataFrame, duckdb.DuckDBPyRelation, pyarrow Table, Dataset, RecordBatchReader, Scanner, or NumPy ndarrays with supported format